In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles

In [2]:
database = 'enigma_1189_sqlite.db'
opsdb = db.OpsimDatabase(database)
outDir = 'meta'
resultsDb = db.ResultsDb(outDir=outDir)

In [13]:
filters = ['u','g','r','i','z','y']
nside = 16
bundleList = []
ss = metrics.MedianMetric()
met = metrics.Coaddm5Metric()
amMet = metrics.MedianMetric(col='airmass')
slicer = slicers.HealpixSlicer(nside=nside)
for f in filters:
    bundle = metricBundles.MetricBundle(met,slicer, 'filter="%s"' % f, summaryMetrics=[ss])
    bundleList.append(bundle)
    bundle = metricBundles.MetricBundle(amMet,slicer, 'filter="%s"' % f, summaryMetrics=[ss])
    bundleList.append(bundle)
    

Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes


In [14]:
bdict = metricBundles.makeBundlesDictFromList(bundleList)
bgroup = metricBundles.MetricBundleGroup(bdict, opsdb, outDir=outDir,
                                         resultsDb=resultsDb)
bgroup.runAll()
#bgroup.plotAll(closefigs=False)

Querying database with constraint filter="y"
Found 446281 visits
Running:  ['opsim_Median_airmass_y_HEAL', 'opsim_CoaddM5_y_HEAL']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database with constraint filter="r"
Found 544710 visits
Running:  ['opsim_Median_airmass_r_HEAL', 'opsim_CoaddM5_r_HEAL']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database with constraint filter="g"
Found 249108 visits
Running:  ['opsim_CoaddM5_g_HEAL', 'opsim_Median_airmass_g_HEAL']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database with constraint filter="u"
Found 186607 visits
Running:  ['opsim_Median_airmass_u_HEAL', 'opsim_CoaddM5_u_HEAL']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database with constraint filter="i"
Found 545488 visits
Running:  ['opsim_Median_airmass_i_HEAL', '

In [16]:
bundleList[2].summaryValues

{'Median': 27.224228918595344}

In [17]:
def baseCompareMetric(object):
    def __init__(self, normVal=None, reverse=False, compareMethod=operator.div):
        """
        Set up a metric value for comparing between runs
        """
        self.normVal = normVal
        self.reverse = reverse
        self.compareMethod = compareMethod
        
    def _compute(self, data):
        """
        Take the data and do some stuff
        """
        # By default, just pass things through.
        return data
        
    def __call__(self, data, raw=False):
        """
        
        """
        result = self._compute(data)
        if reverse:
            result = 1./reverse
        if not raw:
            if normVal is not None:
                result = compareMethod(result,self.normVal)
        return result
        
    

So what we really need, is a list of functions that will operate on the numpy array of the restored summary metric values.  Then we make an array to hold the results that's nRuns x nFunctions in size. Then can plot any way we like!
